<a href="https://colab.research.google.com/github/Musab678/LDA/blob/main/final_business_ass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git


In [ ]:
from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('HUG_KEY'))  # Use your HF Token


In [ ]:
# Import necessary libraries
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=userdata.get('HUG_KEY'),
)

==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# Apply LoRA for Efficient Fine-Tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a business expert with advanced knowledge in Accounting, Banks and Banking, Business, Business Communication, Career management, E-Business, Economics, Entrepreneurship, Finance, Financial Planning, Industries and Professions, Investments, Management and Leadership, Marketing and Sales, Real Estate, Stock Trading, Tax, Small Business, and Entrepreneurship.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""


In [ ]:
# Define train_prompt_style and EOS_TOKEN before using them
train_prompt_style = "### Instruction: {0}\n### Input: {1}\n### Response: {2}"  # Example format
EOS_TOKEN = "<|endoftext|>"  # Or any other suitable end-of-sequence token

def formatting_prompts_func(examples):
    inputs = examples["prompt"]  # Use correct column name
    outputs = examples["completion"]  # Use correct column name
    texts = []

    for input_text, output_text in zip(inputs, outputs):
        text = train_prompt_style.format(input_text, "", output_text) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/894 [00:00<?, ? examples/s]

In [ ]:
# Fine-Tune Model
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=300,  # Increased for better fine-tuning
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

# Start Training
trainer.train()

Map:   0%|          | 0/894 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/894 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/894 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/894 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 894 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 300
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,2.933600
20,2.145500
30,1.981100
40,1.754000
50,1.742000
60,1.406800
70,1.604200
80,1.616200
90,1.321700
100,1.462500


TrainOutput(global_step=300, training_loss=1.1637549591064453, metrics={'train_runtime': 1132.6497, 'train_samples_per_second': 2.119, 'train_steps_per_second': 0.265, 'total_flos': 7710212116758528.0, 'train_loss': 1.1637549591064453})

In [ ]:
train_prompt_style = """### Question:
{0}

### Response:
<think>
{1}
</think>
{2}"""
### Business Solution:"""  # Removed {2} to ensure clean generation

def is_business_related(question):
    # Define expanded keywords for various business-related categories
    business_keywords = [
        "accounting", "bookkeeping", "audit", "financial statement", "ledgers", "tax filing", "balance sheet", "revenue",
        "expenses", "profit", "liabilities", "assets", "accounts receivable", "accounts payable",  # Accounting
        "bank", "banking", "loans", "credit", "savings", "interest rates", "mortgage", "account management", "transactions",
        "bank fees", "digital banking", "financial institution", "deposit", "withdrawal", "business", "sales", "strategy", "growth", "payment", "improvement", "revenue", "startup", "operations",
        "entrepreneurship", "management", "finance", "marketing", "sales", "investment", "market", "payment gateway",  # Banks and Banking
        "business", "strategy", "growth", "startup", "operations", "team management", "business model", "venture",
        "innovation", "profit", "scale", "market share", "industry", "competition",  # Business
        "communication", "email", "meeting", "negotiation", "public relations", "corporate communication", "branding",
        "presentations", "messaging",  # Business Communication
        "career", "job search", "resume", "skills", "job interview", "career development", "promotion", "networking",
        "professional growth", "career path",  # Career Management
        "e-commerce", "online store", "digital marketing", "online business", "website", "payment gateway", "online transaction",
        "digital sales", "product listings", "dropshipping", "affiliate marketing", "e-business model",  # E-Business
        "economics", "market", "supply and demand", "inflation", "GDP", "trade", "fiscal policy", "monetary policy",
        "recession", "growth", "economic theory", "economic trends", "market equilibrium",  # Economics
        "entrepreneur", "startup", "business plan", "funding", "innovation", "pitching", "venture capital", "small business",
        "founder", "leadership", "scalability", "risk-taking", "idea generation",  # Entrepreneurship
        "finance", "investments", "capital", "assets", "liabilities", "equity", "risk", "portfolio", "stock market",
        "bonds", "savings", "returns", "financial analysis", "wealth management",  # Finance
        "financial planning", "budgeting", "savings plan", "retirement", "financial goals", "investment strategies",
        "debt management", "financial advisor", "wealth planning", "estate planning", "tax planning",  # Financial Planning
        "industry", "profession", "career", "job market", "manufacturing", "technology", "healthcare", "retail", "education",
        "service sector", "labor market", "sector", "business professional",  # Industries and Professions
        "investment", "stocks", "bonds", "mutual funds", "ETF", "portfolio management", "risk assessment", "diversification",
        "asset allocation", "real estate investment", "venture capital", "angel investors",  # Investments
        "management", "leadership", "team management", "project management", "decision-making", "strategic planning",
        "organizational structure", "performance", "coaching", "mentoring",  # Management and Leadership
        "marketing", "sales", "advertising", "branding", "content marketing", "SEO", "SEM", "social media", "customer acquisition",
        "lead generation", "market research", "sales funnel", "digital marketing",  # Marketing and Sales
        "real estate", "property", "investment property", "rental", "leasing", "market trends", "commercial real estate",
        "residential property", "real estate agent", "real estate market", "home buying", "property management",  # Real Estate
        "stock trading", "shares", "portfolio", "stock market", "day trading", "brokerage", "stock analysis", "trading strategy",
        "market trends", "investment", "financial markets", "equity", "risk management",  # Stock Trading
        "tax", "taxation", "income tax", "corporate tax", "tax filing", "tax returns", "deductions", "tax planning", "tax strategy",
        "VAT", "tax bracket", "corporate tax rate", "tax laws", "compliance",  # Tax
        "small business", "entrepreneur", "startup", "business plan", "market research", "funding", "business development",
        "small business strategy", "scaling", "operations", "cash flow", "small business management", "business ownership"  # Small Business and Entrepreneurship
    ]
    return any(keyword in question.lower() for keyword in business_keywords)


# Define the question
question = "tellme best stock market techniques"

# Check if the question is business-related
if not is_business_related(question):
    response = "Sorry, I can only generate business solutions. Please provide a business-related question or challenge."
else:
    # Format the prompt
    formatted_prompt = train_prompt_style.format(question, "Analyzing the issue and generating possible solutions.", "Here is the solution to improve your payment gateway and sales globally.")

    # Tokenize the input
    inputs = tokenizer([formatted_prompt], return_tensors="pt").to("cuda")

    # Generate response
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=500,
        use_cache=True,
    )

    # Decode and print the response
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Output the final response
print(response)


### Question:
tellme best stock market techniques

### Response:
<think>
Analyzing the issue and generating possible solutions.
</think>
Here is the solution to improve your payment gateway and sales globally. invest in a robust payment gateway system that supports multiple currencies and localization. develop a global sales strategy that includes localized marketing campaigns and customer support. utilize data analytics to track payment failures and identify areas for improvement. consider partnering with a cloud-based payment processing provider to streamline your payment processes and reduce costs. prioritize building a strong brand presence in emerging markets and focus on cross-border e-commerce opportunities.stock trading and investing are complex and can lead to significant losses if not approached carefully. consider working with a financial advisor or using robo-advisory services to create a personalized investment plan. diversify your portfolio and consider using technical an

In [ ]:
# Save the trained model
model.save_pretrained("deepseek-business-solution-think")
tokenizer.save_pretrained("deepseek-business-solution-think")



('deepseek-business-solution/tokenizer_config.json',
 'deepseek-business-solution/special_tokens_map.json',
 'deepseek-business-solution/tokenizer.json')

In [ ]:
# Push to Hugging Face
model.push_to_hub("your_repo_name", token=userdata.get('HUG_KEY'))
tokenizer.push_to_hub("your_repo_name")


README.md:   0%|          | 0.00/630 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/your_repo_name


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: fineGr

In [ ]:
!git clone https://huggingface.co/Musab123456/business_assistant

Cloning into 'business_assistant'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 1.13 KiB | 1.13 MiB/s, done.


In [ ]:
from huggingface_hub import create_repo, upload_folder

# Define the model path and repository name
repo_name = "business_assistant1"  # Removed trailing space
model_path = "/content/deepseek-business-solution"  # Replace with the path to your saved model folder

# Create a repository on Hugging Face (if not created already)
create_repo(repo_name, private=False)

# Upload the model folder to Hugging Face
upload_folder(
    folder_path=model_path,
    repo_id=f"Musab123456/{repo_name}"  # Replace with your Hugging Face username and repo name
)

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Musab123456/business_assistant1/commit/1d35ec56edfc113f66d3cfe4ba1c4ec4d27b96ec', commit_message='Upload folder using huggingface_hub', commit_description='', oid='1d35ec56edfc113f66d3cfe4ba1c4ec4d27b96ec', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Musab123456/business_assistant1', endpoint='https://huggingface.co', repo_type='model', repo_id='Musab123456/business_assistant1'), pr_revision=None, pr_num=None)

In [ ]:
import shutil

# Create a zip file of the model folder
shutil.make_archive("deepseek-business-solution", 'zip', "deepseek-business-solution")


'/content/deepseek-business-solution.zip'

In [ ]:
from google.colab import files

# Download the zip file
files.download('deepseek-business-solution.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>